In [ ]:
from pprint import pprint

from signac import Collection
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

%matplotlib inline

In [ ]:
with Collection.open('benchmark.txt') as c:
    docs = list(c)

In [ ]:
def strip_complexity(cat):
    assert cat[1] == '_'
    c = cat[0]
    return c, cat[2:]

def normalize(data, N):
    for cat, x in data.items():
        cplx, cat_ = strip_complexity(cat)
        x_mean = min([(y/n) for n, y in x])
        if cplx == 'N':
            x_mean /= N
        if cat_ == 'determine_len':
            x_mean *= 100
        yield cat, 1e3 * x_mean

        
df_meta = pd.DataFrame({doc['_id']: doc['meta'] for doc in docs}).T
df_data = pd.DataFrame({doc['_id']: dict(normalize(doc['data'], doc['meta']['N'])) for doc in docs}).T
df = pd.concat([df_meta, df_data], axis=1)

In [ ]:
def tr(s):
    return {
        '1_select_by_id': "Select by ID O(1)",
        'N_determine_len': "Determine N (x100) O(N)",
        'N_iterate': "Iterate (multiple passes) O(N)",
        'N_iterate_single_pass': "Iterate (single pass) O(N)",
        'N_search_lean_filter': "Search w/ lean filter O(N)",
        'N_search_rich_filter': "Search w/ rich filter O(N)",
        'datreant.core': "datreant",
        'tool,N': "Tool, N",
    }.get(s, s)

def tr_legend(legend):
    title = legend.get_title()
    title.set_text(tr(title.get_text()))
    for text in legend.get_texts():
        tool, N = text.get_text()[1:-1].split(',')
        text.set_text("{}, {}".format(tr(tool), N))

fig, ax = plt.subplots(figsize=(4, 3), dpi=150)

cmap_signac = plt.get_cmap('Blues')
cmap_datreant = plt.get_cmap('Greens')


def cmap(cmaps, a, x):
    n = len(cmaps)
    i = (n+1) * x
    j = int(i / n)
    y = (i / n - int(i/n)) * (1-a) + a
    return cmaps[int(i/n)](y)
    return cmap_signac(x)
    
from functools import partial
colors = partial(cmap, [cmap_datreant, cmap_signac], 0.2)

    
df.rename(columns=tr).groupby(by=['tool', 'N']).mean().T.plot(kind='barh', ax=ax, colormap=colors)
legend = ax.get_legend()
tr_legend(legend)
ax.set_xlim(0, 10.0)
ax.set_xlabel('Time / Complexity [ms]')
plt.show()